In [1]:
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
import requests

import transformers
import os
import re
from collections import Counter
from transformers import BertConfig, BertModel, BertForMaskedLM, BertTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import sys
import torch# Convert data to PyTorch tensors
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
import random
print(sys.version)
print(os.getcwd())

# Load the pre-trained BERT tokenizer and model for masked language modeling
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/home/qqi2/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.8.10 (default, Sep 11 2024, 16:02:53) 
[GCC 9.4.0]
/data/wiki052024/code


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [10]:
#pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached datasets-3.0.1-py3-none-any.whl (471 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 116.6 MB/s eta 0:00

In [2]:
#pip install scipy


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 176.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 131.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached huggingface_hub-0.25.1-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 115.4 MB/s eta 0:00:00
Using cached huggingface_hub-0.25.1-py3-none-any.whl (436 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.0/785.0 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 88.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# initialize the pipeline
# here we choose whether to use gpu or cpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # prefer GPU if available
    print('Torch on CUDA')
else:
    device = torch.device("cpu")  # CPU if not
    print('Torch on CPU')
model = model.to(device)

# Load your corpus
with open('/data/wiki052024/data/he_wiki_cleaned_sample.txt', 'r', encoding='utf-8') as f:
    corpus_file_sample = [line.strip() for line in f if line.strip()]
print(type(corpus_file_sample))
print(len(corpus_file_sample))
print(corpus_file_sample[0:10])

Torch on CUDA
<class 'list'>
28916
['müller had a strong start to the grass court season reaching the semifinals of the topshelf open as well as the quarterfinals at queens beating defending champion dimitrov for the second time in two meetings since the beginning of the year', 'the newspaper has had a circulation reaching', 'these two aircraft also proved underpowered and the type was abandoned with the last twin seaplane written off in', 'the diocese is a member of the conference of the latin bishops of the arab regions', 'because the early jurchens had no written records the story of hanpu was first transmitted orally', 'both clubs grew in stature as the s progressed leading to their first meeting of the s being billed as a meeting of two local clubs ardwick city just rising to the highest standard in the football world whilst newton heath are nearly in the front rank', 'after completing high school he entered syracuse university where he graduated in', 'some of these gaelic songs a

In [3]:
# 3. Convert sentences to a Hugging Face Dataset
dataset = Dataset.from_dict({"text": corpus_file_sample})

max_length = 128  # Set this according to your needs
# Tokenize the texts with padding and truncation
# 4. Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=max_length)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Output encoded texts
print("Token IDs:", tokenized_datasets['input_ids'])
print("Attention Masks:", tokenized_datasets['attention_mask'])


# 5. Data Collator for Masked Language Modeling (MLM)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15  # Probability of masking a token
)

training_args = TrainingArguments(
    output_dir="data/wiki052024/data/bert/bert-mlm-finetune",  # Directory to save the model checkpoints
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01
)

# 7. Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,  # Training data
    tokenizer=tokenizer
)

# 8. Fine-tune the model
trainer.train()

# 9. Save the fine-tuned model
model.save_pretrained("data/wiki052024/data/bert/bert-mlm-finetune")
tokenizer.save_pretrained("data/wiki052024/data/bert/bert-mlm-finetune")

Map: 100%|██████████| 28916/28916 [00:13<00:00, 2187.20 examples/s]
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Step,Training Loss
500,2.854000
1000,2.675000
1500,2.589200
2000,2.515000
2500,2.481800


('data/wiki052024/data/bert/bert-mlm-finetune/tokenizer_config.json',
 'data/wiki052024/data/bert/bert-mlm-finetune/special_tokens_map.json',
 'data/wiki052024/data/bert/bert-mlm-finetune/vocab.txt',
 'data/wiki052024/data/bert/bert-mlm-finetune/added_tokens.json')

In [16]:
#Although it shows '/data/wiki052024/data/bert/bert-mlm-finetune/added_tokens.json'. However, added_tokens.json is not created. 
model.save_pretrained("/data/wiki052024/data/bert/bert-mlm-finetune")
tokenizer.save_pretrained("/data/wiki052024/data/bert/bert-mlm-finetune")

('/data/wiki052024/data/bert/bert-mlm-finetune/tokenizer_config.json',
 '/data/wiki052024/data/bert/bert-mlm-finetune/special_tokens_map.json',
 '/data/wiki052024/data/bert/bert-mlm-finetune/vocab.txt',
 '/data/wiki052024/data/bert/bert-mlm-finetune/added_tokens.json')